# Web Scraping Indeed.com for job IDs and keyword

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# String format: "keyword keyword"
def count_results(query_string = None):
    if query_string == None:
        return(print("No keyword entered."))
    
    job_ids = pd.DataFrame()
    
    result_list = []
        
    # Find the number of results
    URL_for_count = "http://www.indeed.com/jobs?q=data+scientist+%28{}%29&l=Boston".format(query_string)
    soup_for_count = BeautifulSoup(urlopen(URL_for_count).read(), 'html.parser')

    results_number = soup_for_count.find("div", attrs = {"id": "searchCount"}).text
    number_of_results = int(results_number.split(sep = ' ')[-1].replace(',', ''))

    # Now loop through the pages. Viewing 100 results at a time means fewer page refreshes.
    i = int(number_of_results/100)
    for page_number in range(i + 1):
        URL_for_results = "http://www.indeed.com/jobs?q=data+scientist+%28{}%29&l=Boston&limit=100&start={}".format(query_string, str(100 * page_number))
        soup_for_results = BeautifulSoup(urlopen(URL_for_results).read(), 'html.parser')
        results = soup_for_results.find_all('div', attrs={'data-tn-component': 'organicJob'})
            
        # Extract the ID for each job listing, and include the 'salary_over_X' level
        for x in results:
            result_list.append([x.find('h2', attrs={"class": "jobtitle"})['id'], 1])
        
        # Add the job ID numbers
        job_ids = job_ids.append(result_list)
    
    # Rename job_ids's columns
    job_ids.columns = ['id', "{}".format(query_string)]
    
    # Remove re-posted jobs
    job_ids.drop_duplicates(inplace = True)
    
    job_ids.to_csv(path_or_buf="id_and_{}.csv".format(query_string))

In [3]:
count_results()

No keyword entered.
